In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
from efficientnet_pytorch import EfficientNet
from dataset.MicroscopyTrainDataLoader import MicroscopyTrainDataLoader
from experiments.microscopy.microscopy import experiment, get_model

LABELS_PATH = '../labels/microscopy.csv'

In [5]:
model = get_model("efficient-net", "efficient-net", num_classes=4, pretrained=True)
checkpoint = torch.load('../outputs/{0}/checkpoint.pt'.format("deft-sound-398"))
model.load_state_dict(checkpoint)

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [6]:
loader = MicroscopyTrainDataLoader(LABELS_PATH)
test_dataset = loader.get_test_dataset()

In [10]:
import torch.nn as nn
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

kwargs = {'num_workers': 16, 'pin_memory': True} if torch.cuda.is_available() else {}
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, **kwargs)

y_true = []
x = []
y_pred = []

test_loss, n_iter = 0, 0
total, correct = 0, 0

for images, labels, img_path in test_loader:
    y_true += labels
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    y_pred += predicted.cpu()
    
    total += labels.size(0)
    correct += torch.sum(predicted == labels.data)
    n_iter += 1
    
    acc = 100 * torch.true_divide(correct, total)
    
    target_names = test_dataset.codec.classes_

print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        DMEL       0.59      0.51      0.55        88
        DMFL       0.94      0.93      0.94       284
        DMLI       0.93      0.96      0.95       405
        DMTR       0.73      0.75      0.74        96

    accuracy                           0.88       873
   macro avg       0.80      0.79      0.79       873
weighted avg       0.88      0.88      0.88       873



In [11]:
from sklearn.metrics import accuracy_score

In [12]:
accuracy_score(y_true, y_pred)

0.8831615120274914

In [14]:
from sklearn.metrics import confusion_matrix
import numpy as np

cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
cm.diagonal()

array([0.51136364, 0.93309859, 0.96049383, 0.75      ])

In [42]:
import pandas as pd

classes = [y.item() for y in y_true]
classes = test_dataset.codec.inverse_transform(classes)

df = pd.DataFrame(np.array(classes).reshape(len(classes), 1))

In [45]:
df.columns = ['y']
df = df.reset_index()
df.head()

,index,y
0,0,DMEL
1,1,DMEL
2,2,DMEL
3,3,DMEL
4,4,DMEL


In [46]:
df.groupby('y').count()

,index
y,
DMEL,88
DMFL,284
DMLI,405
DMTR,96


In [47]:
test_dataset.codec.classes_

array(['DMEL', 'DMFL', 'DMLI', 'DMTR'], dtype='<U4')